In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

# 1. 读取并处理输入文件，将文本全部转换为小写
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

# 2. 创建字符到整数的映射
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# 3. 输出文本的总字符数和独特字符数
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# 4. 准备数据集，根据设定的序列长度（seq_length）将输入文本转换为整数编码的输入输出对
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# 5. 转换并标准化输入数据为PyTorch张量
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)

# 6. 定义一个字符级LSTM模型，其中包括一个LSTM层、一个Dropout层以及一个全连接层
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.linear(self.dropout(x))
        return x

# 7. 设置训练参数：迭代次数（n_epochs）和批量大小（batch_size）
n_epochs = 40
batch_size = 128

# 8. 初始化模型并将其移到GPU上（如果可用）
model = CharModel()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 9. 选择优化器（Adam）和损失函数（交叉熵损失）
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=batch_size)

# 10. 训练模型，同时记录最佳模型状态
best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # 11. 验证模型性能，如果性能更好，记录最佳模型
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.to(device))
            loss += loss_fn(y_pred, y_batch.to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))

# 12. 保存最佳模型状态和字符到整数的映射
torch.save([best_model, char_to_int], "single-char.pth")

# 13. 加载保存的模型状态和字符到整数的映射
best_model, char_to_int = torch.load("single-char.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model.load_state_dict(best_model)

# 14. 随机选择一个文本片段作为生成文本的初始提示
filename = "wonderland.txt"
seq_length = 100
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]
pattern = [char_to_int[c] for c in prompt]

# 15. 使用训练好的模型生成文本
model.eval()
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    for i in range(1000):
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        prediction = model(x.to(device))
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")


Total Characters:  164014
Total Vocab:  63
Total Patterns:  163914
Epoch 0: Cross-entropy: 438310.6250
Epoch 1: Cross-entropy: 404915.1875
Epoch 2: Cross-entropy: 378032.7188
Epoch 3: Cross-entropy: 358118.6562
Epoch 4: Cross-entropy: 340549.2500
Epoch 5: Cross-entropy: 326148.2812
Epoch 6: Cross-entropy: 315271.2812
Epoch 7: Cross-entropy: 304081.3750
Epoch 8: Cross-entropy: 295791.5000
Epoch 9: Cross-entropy: 288530.8438
Epoch 10: Cross-entropy: 285020.3750
Epoch 11: Cross-entropy: 274777.7188
Epoch 12: Cross-entropy: 267050.9062
Epoch 13: Cross-entropy: 263274.4375
Epoch 14: Cross-entropy: 259068.4688
Epoch 15: Cross-entropy: 253655.7656
Epoch 16: Cross-entropy: 248130.1562
Epoch 17: Cross-entropy: 245956.7500
Epoch 18: Cross-entropy: 240772.8594
Epoch 19: Cross-entropy: 238264.5781
Epoch 20: Cross-entropy: 235628.5000
Epoch 21: Cross-entropy: 232212.8125
Epoch 22: Cross-entropy: 226408.9531
Epoch 23: Cross-entropy: 225582.9219
Epoch 24: Cross-entropy: 220097.8281
Epoch 25: Cross-en